In [1]:
from PIL import Image
from numpy import asarray

image = Image.open('/content/9911_16x16_icon.png')

# Convert the image to RGB format
rgb_image = image.convert('RGB')

# Convert the RGB image to numpy array
numpydata = asarray(rgb_image)

numpydata

print(numpydata)
print(numpydata.shape)

[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0  93   0]
  [  0  93   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0  93   0]
  [255 255   0]
  [  0  93   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]
  [  0  93   0]
  [255 255   0]
  

In [2]:
import numpy as np

def rgb_to_ycbcr(rgb_array):
    # Convert RGB to YCbCr
    y_array = 0.299 * rgb_array[:, :, 0] + 0.587 * rgb_array[:, :, 1] + 0.114 * rgb_array[:, :, 2]
    cb_array = 128 - 0.168736 * rgb_array[:, :, 0] - 0.331264 * rgb_array[:, :, 1] + 0.5 * rgb_array[:, :, 2]
    cr_array = 128 + 0.5 * rgb_array[:, :, 0] - 0.418688 * rgb_array[:, :, 1] - 0.081312 * rgb_array[:, :, 2]

    return y_array.astype(np.uint8), cb_array.astype(np.uint8), cr_array.astype(np.uint8)

# Assuming numpydata contains the RGB image array
rgb_image = numpydata

# Convert RGB image to YCbCr format
y_array, cb_array, cr_array = rgb_to_ycbcr(rgb_image)

print("Y array:")
print(y_array)
print(y_array.shape)

print("\nCb array:")
print(cb_array)
print(cb_array.shape)

print("\nCr array:")
print(cr_array)
print(cr_array.shape)


Y array:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  54  54]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  54 225  54]
 [  0   0   0   0   0   0   0   0   0   0   0   0  54 225 144  54]
 [  0   0   0   0   0   0   0   0   0   0   0  54 225 144 109  54]
 [  0   0   0   0   0   0   0   0   0   0  54 225 144 109  98  54]
 [  0   0   0   0   0   0   0   0   0  54 225 144 109  98 125  54]
 [  0   0  54  54   0   0   0   0  54 225 144 109  98 125  41   0]
 [  0  54 144 225  54   0   0  54 225 144 109  98 125  41   0   0]
 [ 54  98 109 144 225  54  54 225 144 109  98 125  41   0   0   0]
 [ 54 125  98 109 144 225 225 144 109  98 125  41   0   0   0   0]
 [  0  41 125  98 109 144 144 109  98 125  41   0   0   0   0   0]
 [  0   0  41 125  98 109 109  98 125  41   0   0   0   0   0   0]
 [  0   0   0  42 133  98  98 125  41   0   0   0   0   0   0   0]
 [  0   0   0   0  42 125 125  41   0   0   0   0   0

In [3]:
def downsample_channel(channel):
    """
    Downsamples a given channel by averaging neighboring pixels into larger blocks.
    """
    height, width = channel.shape
    # New dimensions after downsampling (divide by 2 in each dimension)
    new_height, new_width = height // 2, width // 2
    downsampled_channel = np.zeros((new_height, new_width), dtype=np.uint8)
    for i in range(new_height):
        for j in range(new_width):
            # Average neighboring 2x2 pixels
            downsampled_channel[i, j] = np.mean(channel[i*2:i*2+2, j*2:j*2+2])
    return downsampled_channel

In [4]:
def divide_into_blocks(image, block_size=8):
    """
    Divides the image into smaller blocks of specified size.
    """
    height, width = image.shape
    # Calculate the number of blocks in each dimension
    num_blocks_height = height // block_size
    num_blocks_width = width // block_size
    blocks = []
    for i in range(num_blocks_height):
        for j in range(num_blocks_width):
            # Extract block from the image
            block = image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            blocks.append(block)
    return np.array(blocks)

In [6]:
# Downsampling chrominance channels (Cb and Cr)
downsampled_cb = downsample_channel(cb_array)
downsampled_cr = downsample_channel(cr_array)

# Division into 8x8 Pixel Blocks
y_blocks_8x8 = divide_into_blocks(y_array)

print("Dimensions after downsampling:")
print("Cb channel:", downsampled_cb)
print("Cb channel:", downsampled_cb.shape)
print("Cr channel:", downsampled_cr.shape)
print("Dimensions of Divided Y blocks:", y_blocks_8x8.shape)

print(y_blocks_8x8)

Dimensions after downsampling:
Cb channel: [[128 128 128 128 128 128 128 112]
 [128 128 128 128 128 128  80  65]
 [128 128 128 128 128  80  53  86]
 [128 112 128 128  80  53  74 100]
 [ 99  53  80  80  53  74 102 128]
 [100  74  53  53  74 102 128 128]
 [128 102  75  74 102 128 128 128]
 [128 128 102 102 128 128 128 128]]
Cb channel: (8, 8)
Cr channel: (8, 8)
Dimensions of Divided Y blocks: (4, 8, 8)
[[[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0]
  [  0   0  54  54   0   0   0   0]]

 [[  0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0  54  54]
  [  0   0   0   0   0  54 225  54]
  [  0   0   0   0  54 225 144  54]
  [  0   0   0  54 225 144 109  54]
  [  0   0  54 225 144 109  98  54]
  [  0  54 225 144 109  98 125  54]
  [ 54 225 144 109  98 125  41   0]]

 [[  0  54 144 2

DCT matrix multiplication

In [7]:
import numpy as np

def dct2_matrix_multiplication(matrix):
    """Compute 2D Discrete Cosine Transform using matrix multiplication"""

    a = 1.387039
    b = 1.306563
    c = 1.175876
    d = 0.785695
    e = 0.541196
    f = 0.275899

    A_t = np.array([[1, 1, 1, 1, 1, 1, 1, 1],
                    [a, c, d, f, -f, -d, -c, -a],
                    [b, e, -e, -b, -b, -e, e, b],
                    [c, -f, -a, -d, d, a, f, -c],
                    [1, -1, -1, 1, 1, -1, -1, 1],
                    [d, -a, f, c, -c, -f, a, -d],
                    [e, -b, b, -e, -e, b, -b, e],
                    [f, -d, c, -a, a, -c, d, -f]])

    A = np.array([[1, a, b, c, 1, d, e, f],
                  [1, c, e, -f, -1, -a, -b, -d],
                  [1, d, -e, -a, -1, f, b, c],
                  [1, f, -b, -d, 1, c, -e, -a],
                  [1, -f, -b, d, 1, -c, -e, a],
                  [1, -d, -e, a, -1, -f, b, -c],
                  [1, -c, e, f, -1, a, -b, d],
                  [1, -a, b, -c, 1, -d, e, -f]])


    # Compute DCT coefficients using matrix multiplication
    dct_result = A_t.dot(matrix).dot(A) / 8
    # print(A_t.dot(matrix))

    return dct_result

# DCT applied on Y, CB, Cr 8 X 8
dct_Y = y_blocks_8x8[0]

# Compute 2D DCT using matrix multiplication
dct_result_y = dct2_matrix_multiplication(dct_Y)
dct_result_cb = dct2_matrix_multiplication(downsampled_cb)
dct_result_cr = dct2_matrix_multiplication(downsampled_cr)

# Print original matrix and DCT result

# print("Original 8x8 Matrix:")
# print(example_matrix)
# print()
print("for y")
print(dct_result_y)
print("for cb")
print(dct_result_cb)
print("for cr")
print(dct_result_cr)

for y
[[ 13.5          7.1657595  -12.47237325 -14.6659545    0.
    9.79948125   5.16622725  -1.42535025]
 [-18.7250265   -9.93918789  17.29966812  20.34225086   0.
  -13.59226267  -7.16575868   1.97701639]
 [ 17.6386005    9.36251623 -16.29594141 -19.16199351   0.
   12.80363962   6.75000137  -1.8623099 ]
 [-15.874326    -8.42604462  14.66596437  17.24534391   0.
  -11.52297481  -6.07484263   1.67603515]
 [ 13.5          7.1657595  -12.47237325 -14.6659545    0.
    9.79948125   5.16622725  -1.42535025]
 [-10.6068825   -5.63010141   9.7994813   11.52296712   0.
   -7.69940342  -4.05907892   1.11989056]
 [  7.306146     3.87808038  -6.74999851  -7.93715591   0.
    5.30344005   2.79594152  -0.77139385]
 [ -3.7246365   -1.97702588   3.44111531   4.04632218   0.
   -2.70366708  -1.42535693   0.39325271]]
for cb
[[ 8.50125000e+02  2.07526644e+01  3.47612095e+01  1.22455981e+01
   2.08750000e+01  3.52182388e+00  8.46696837e+00  1.77043000e-01]
 [ 3.23553854e+01  8.23347250e+01 -7.09251750

In [8]:
import numpy as np

# Define quantization matrix for luminance (Y) component
quantization_matrix_y = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                  [12, 12, 14, 19, 26, 58, 60, 55],
                                  [14, 13, 16, 24, 40, 57, 69, 56],
                                  [14, 17, 22, 29, 51, 87, 80, 62],
                                  [18, 22, 37, 56, 68, 109, 103, 77],
                                  [24, 35, 55, 64, 81, 104, 113, 92],
                                  [49, 64, 78, 87, 103, 121, 120, 101],
                                  [72, 92, 95, 98, 112, 100, 103, 99]])

# Define quantization matrices for chrominance (Cb and Cr) components
quantization_matrix_cbcr = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                     [18, 21, 26, 66, 99, 99, 99, 99],
                                     [24, 26, 56, 99, 99, 99, 99, 99],
                                     [47, 66, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99]])
def quantize(matrix, quantization_matrix):
    """
    Perform quantization on an 8x8 matrix.
    """
    quantized_matrix = np.round(matrix / quantization_matrix)
    # Handle negative zero
    quantized_matrix[np.isclose(quantized_matrix, 0, atol=1e-10)] = 0
    return quantized_matrix


# Apply quantization on DCT coefficients
quantized_y = quantize(dct_result_y, quantization_matrix_y)
quantized_cb = quantize(dct_result_cb, quantization_matrix_cbcr)
quantized_cr = quantize(dct_result_cr, quantization_matrix_cbcr)

# Print quantized matrices
print("Quantized Y matrix:")
print(quantized_y)
print()
print("Quantized Cb matrix:")
print(quantized_cb)
print()
print("Quantized Cr matrix:")
print(quantized_cr)


Quantized Y matrix:
[[ 1.  1. -1. -1.  0.  0.  0.  0.]
 [-2. -1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1. -1. -1.  0.  0.  0.  0.]
 [-1.  0.  1.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]

Quantized Cb matrix:
[[50.  1.  1.  0.  0.  0.  0.  0.]
 [ 2.  4. -3.  0.  0.  0.  0.  0.]
 [ 4.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]

Quantized Cr matrix:
[[53.  1.  1.  0.  0.  0.  0.  0.]
 [ 2.  3. -2.  0.  0.  0.  0.  0.]
 [ 2. -1.  0.  0.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]


In [9]:
def zigzag(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    result = []

    for i in range(rows + cols - 1):
        if i % 2 == 0:  # Even-indexed diagonals (downward)
            if i < rows:  # Diagonals starting from the top row
                for j in range(min(i + 1, cols)):
                    result.append(matrix[i - j][j])
            else:  # Diagonals starting from the last column
                for j in range(i - rows + 1, rows):
                    result.append(matrix[j][cols - 1 - (i - rows + j)])
        else:  # Odd-indexed diagonals (upward)
            if i < cols:  # Diagonals starting from the first column
                for j in range(min(i + 1, rows)):
                    result.append(matrix[j][i - j])
            else:  # Diagonals starting from the last row
                for j in range(i - cols + 1, cols):
                    result.append(matrix[rows - 1 - (i - cols + j)][j])

    return result



# Obtain zigzag traversal for each matrix
zigzag_result_y = zigzag(quantized_y)
zigzag_result_cb = zigzag(quantized_cb)
zigzag_result_cr = zigzag(quantized_cr)

print("Zigzag traversal for Y matrix:")
print(zigzag_result_y)
print()
print("Zigzag traversal for Cb matrix:")
print(zigzag_result_cb)
print()
print("Zigzag traversal for Cr matrix:")
print(zigzag_result_cr)


Zigzag traversal for Y matrix:
[1.0, 1.0, -2.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Zigzag traversal for Cb matrix:
[50.0, 1.0, 2.0, 4.0, 4.0, 1.0, 0.0, -3.0, 0.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Zigzag traversal for Cr matrix:
[53.0, 1.0, 2.0, 2.0, 3.0, 1.0, 0.0, -2.0, -1.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [10]:
def rle(input_list):
    if not input_list:
        return []

    result = []
    current_count = 1

    for i in range(1, len(input_list)):
        if input_list[i] == input_list[i - 1]:
            current_count += 1
        else:
            result.append((input_list[i - 1], current_count))
            current_count = 1

    # Add the last element
    result.append((input_list[-1], current_count))
    return result



# Run-length encoding for Y, Cb, and Cr lists
rle_result_y = rle(zigzag_result_y)
rle_result_cb = rle(zigzag_result_cb)
rle_result_cr = rle(zigzag_result_cr)

print("Run-length encoding for Y list:")
print(rle_result_y)
print()
print("Run-length encoding for Cb list:")
print(rle_result_cb)
print()
print("Run-length encoding for Cr list:")
print(rle_result_cr)


Run-length encoding for Y list:
[(1.0, 2), (-2.0, 1), (1.0, 1), (-1.0, 3), (1.0, 2), (-1.0, 1), (1.0, 1), (0.0, 1), (-1.0, 1), (1.0, 1), (0.0, 3), (-1.0, 1), (1.0, 1), (0.0, 5), (1.0, 1), (0.0, 19), (1.0, 1), (0.0, 4), (1.0, 1), (0.0, 14)]

Run-length encoding for Cb list:
[(50.0, 1), (1.0, 1), (2.0, 1), (4.0, 2), (1.0, 1), (0.0, 1), (-3.0, 1), (0.0, 1), (-1.0, 1), (0.0, 1), (-1.0, 1), (0.0, 52)]

Run-length encoding for Cr list:
[(53.0, 1), (1.0, 1), (2.0, 2), (3.0, 1), (1.0, 1), (0.0, 1), (-2.0, 1), (-1.0, 2), (0.0, 1), (-1.0, 1), (0.0, 52)]


In [12]:
import heapq
from collections import defaultdict, Counter

class Node:
    def __init__(self, freq, value=None):
        self.freq = freq
        self.value = value
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    heap = [Node(freq, value) for value, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = Node(left.freq + right.freq)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0]

def generate_huffman_codes(node, code="", codes=None):
    if codes is None:
        codes = {}
    if node is not None:
        if node.value is not None:
            codes[node.value] = code
        generate_huffman_codes(node.left, code + "0", codes)
        generate_huffman_codes(node.right, code + "1", codes)
    return codes

def huffman_encoding(values):
    frequencies = Counter(values)
    root = build_huffman_tree(frequencies)
    codes = generate_huffman_codes(root)
    encoded_values = [codes[value] for value in values]
    return encoded_values, codes


# Huffman encoding for Y, Cb, and Cr values
encoded_values_Y, huffman_codes_Y = huffman_encoding(rle_result_y)
encoded_values_Cb, huffman_codes_Cb = huffman_encoding(rle_result_cb)
encoded_values_Cr, huffman_codes_Cr = huffman_encoding(rle_result_cr)

print("Encoded values for Y:", encoded_values_Y)
print("Huffman codes for Y:", huffman_codes_Y)
print()
print("Encoded values for Cb:", encoded_values_Cb)
print("Huffman codes for Cb:", huffman_codes_Cb)
print()
print("Encoded values for Cr:", encoded_values_Cr)
print("Huffman codes for Cr:", huffman_codes_Cr)

Encoded values for Y: ['010', '0010', '11', '0111', '010', '101', '11', '0001', '101', '11', '0000', '101', '11', '0110', '11', '0011', '11', '1001', '11', '1000']
Huffman codes for Y: {(0.0, 3): '0000', (0.0, 1): '0001', (-2.0, 1): '0010', (0.0, 19): '0011', (1.0, 2): '010', (0.0, 5): '0110', (-1.0, 3): '0111', (0.0, 14): '1000', (0.0, 4): '1001', (-1.0, 1): '101', (1.0, 1): '11'}

Encoded values for Cb: ['1100', '101', '1111', '100', '101', '01', '1110', '01', '00', '01', '00', '1101']
Huffman codes for Cb: {(-1.0, 1): '00', (0.0, 1): '01', (4.0, 2): '100', (1.0, 1): '101', (50.0, 1): '1100', (0.0, 52): '1101', (-3.0, 1): '1110', (2.0, 1): '1111'}

Encoded values for Cr: ['010', '111', '1011', '100', '111', '00', '011', '1010', '00', '1100', '1101']
Huffman codes for Cr: {(0.0, 1): '00', (53.0, 1): '010', (-2.0, 1): '011', (3.0, 1): '100', (-1.0, 2): '1010', (2.0, 2): '1011', (-1.0, 1): '1100', (0.0, 52): '1101', (1.0, 1): '111'}


In [15]:
import numpy as np
from scipy import fftpack

# Define quantization matrix for luminance (Y) component
quantization_matrix_y = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                  [12, 12, 14, 19, 26, 58, 60, 55],
                                  [14, 13, 16, 24, 40, 57, 69, 56],
                                  [14, 17, 22, 29, 51, 87, 80, 62],
                                  [18, 22, 37, 56, 68, 109, 103, 77],
                                  [24, 35, 55, 64, 81, 104, 113, 92],
                                  [49, 64, 78, 87, 103, 121, 120, 101],
                                  [72, 92, 95, 98, 112, 100, 103, 99]])

# Define quantization matrices for chrominance (Cb and Cr) components
quantization_matrix_cbcr = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                     [18, 21, 26, 66, 99, 99, 99, 99],
                                     [24, 26, 56, 99, 99, 99, 99, 99],
                                     [47, 66, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99],
                                     [99, 99, 99, 99, 99, 99, 99, 99]])

def inverse_zigzag_2d_array(arr):
    m, n = 8, 8
    matrix = [[0] * n for _ in range(m)]
    row, col = 0, 0
    going_up = True

    for num in arr:
        matrix[row][col] = num
        if going_up:
            if row == 0 or col == n - 1:
                going_up = False
                if col == n - 1:
                    row += 1
                else:
                    col += 1
            else:
                row -= 1
                col += 1
        else:
            if col == 0 or row == m - 1:
                going_up = True
                if row == m - 1:
                    col += 1
                else:
                    row += 1
            else:
                row += 1
                col -= 1

    return matrix


def rle_decode(encoded_list):
    result = []
    for value, count in encoded_list:
        result.extend([value] * count)
    return result


def dequantize(matrix, quantization_matrix):
    return matrix * quantization_matrix


def idct2(matrix):
    return fftpack.idct(fftpack.idct(matrix.T, norm='ortho').T, norm='ortho')

def ycbcr_to_rgb(y, cb, cr):
    y = y + 128
    cb = cb + 128
    cr = cr + 128
    r = 1.402 * (cr - 128) + y
    g = -0.34414 * (cb - 128) - 0.71414 * (cr - 128) + y
    b = 1.772 * (cb - 128) + y
    return np.clip(np.stack([r, g, b], axis=-1), 0, 255).astype(np.uint8)

# Decode Huffman encoded values
decoded_values_Y = rle_decode(rle_result_y)
decoded_values_Cb = rle_decode(rle_result_cb)
decoded_values_Cr = rle_decode(rle_result_cr)
# Put the list of numbers into an 8×8 matrix
matrix_y = inverse_zigzag_2d_array(decoded_values_Y)
matrix_cb = inverse_zigzag_2d_array(decoded_values_Cb)
matrix_cr =  inverse_zigzag_2d_array(decoded_values_Cr)
print(matrix_cb)
# Dequantize matrices
dequantized_y = dequantize(matrix_y, quantization_matrix_y)
dequantized_cb = dequantize(matrix_cb, quantization_matrix_cbcr)
dequantized_cr = dequantize(matrix_cr, quantization_matrix_cbcr)
print(dequantized_cb)
# Inverse DCT
idct_y = idct2(dequantized_y)
idct_cb = idct2(dequantized_cb)
idct_cr = idct2(dequantized_cr)

# Convert YCbCr to RGB
final_image = ycbcr_to_rgb(idct_y, idct_cb, idct_cr)
# print(final_image)
# # Display the image
# from PIL import Image
Image.fromarray(final_image).show()

print(idct_cb)


print(final_image)

[[50.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2.0, 4.0, -3.0, 0.0, 0.0, 0.0, 0.0, 0.0], [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[850.  18.  24.   0.   0.   0.   0.   0.]
 [ 36.  84. -78.   0.   0.   0.   0.   0.]
 [ 96.   0.   0.   0.   0.   0.   0.   0.]
 [-47. -66.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]
[[117.37830706 123.93075169 132.54612258 136.97441904 133.04950501
  121.36891307 107.20287552  97.64643175]
 [132.00025917 134.91598126 137.49473599 135.37169225 126.061267
  110.9808881   95.23520371  85.19359064]
 [135.78650577 134.71996708 131.20279849 123.58342948 111.47537786
   96.72198474  83.11578251  74